In [7]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
import numpy as np
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Evaluation.K_FoldEvaluator import K_FoldEvaluator
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeTwoModelsByScores, MergeThreeModelsByScores

In [8]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()

URM_train_list=[]
URM_validation_list=[]
evaluator_list=[]

In [9]:
for k in range(5):
    URM_train,URM_validation = split_train_in_two_percentage_global_sample(URM, train_percentage=0.8)
    URM_train_list.append(URM_train)
    URM_validation_list.append(URM_validation)
    
    ##SELECT 25% OF USERS WITH LONGEST PROFILES TO FINE-TUNE THE HYPERPARAMETERS
    users_to_ignore=[]
    profile_length = np.ediff1d(URM_train.indptr)
    block_size = int(len(profile_length) * 0.25)

    start_pos = 3 * block_size
    end_pos = min(4 * block_size, len(profile_length))
    sorted_users = np.argsort(profile_length)

    users_in_group = sorted_users[start_pos:end_pos]

    users_in_group_p_len = profile_length[users_in_group]
    sorted_users = np.argsort(profile_length)

    users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)
    evaluator_list.append(EvaluatorHoldout(URM_validation, cutoff_list=[10],verbose=False,ignore_users=sorted_users[users_not_in_group_flag]))


 71%|██████████████████████████▏          | 12800/18059 [11:20<02:16, 38.61it/s]

In [10]:
URM_combined_list=[]
for URM_i in URM_train_list:
    URM_combined_list.append(combine_matrices(ICM=ICM,URM=URM_i))

In [11]:
from bayes_opt import BayesianOptimization

In [12]:
rp3_recommenders=[]
slime_recommenders=[]
IALS_recommenders=[]
similarity_recommenders=[]
recommenders=[]


for i in range(len(URM_train_list)):
    IALS_recommenders.append(IALSRecommender(URM_train=URM_combined_list[i]))
    IALS_recommenders[i].fit(epochs=10, 
                             num_factors=42, 
                             alpha=0.7617528864750021, 
                             reg=8.926401306541349)
    
    rp3_recommenders.append(RP3betaRecommender(URM_train=URM_combined_list[i]))
    rp3_recommenders[i].fit(topK=181, 
                            alpha=0.5709402717259106, 
                            beta=0.3566066452521787, 
                            implicit=True, 
                            normalize_similarity=True)
    
    slime_recommenders.append(MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_combined_list[i]))
    slime_recommenders[i].fit(topK=420, 
                              l1_ratio=0.06061216785568925, 
                              alpha=0.00554982938879437, 
                              workers = 20)
    
    sim_slime = slime_recommenders[i].W_sparse
    sim_rp3 = rp3_recommenders[i].W_sparse
    
    similarity_recommenders.append(ItemKNNSimilarityHybridRecommender(URM_train=URM_combined_list[i],
                                                                     Similarity_1=sim_rp3,
                                                                     Similarity_2=sim_slime))
    similarity_recommenders[i].fit(topK=int(524.9285577442362), 
                                   alpha=0.16115285930157153)
    
    
    recommender = MergeTwoModelsByScores(URM_train=URM_combined_list[i],
                                         recommenders=[similarity_recommenders[i],IALS_recommenders[i]])
    recommenders.append(recommender)


IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 7.77 sec
IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 16.69 sec
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 26.76 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 35.80 sec
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 46.17 sec
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 59.77 sec
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 1.16 min
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 1.30 min
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 1.45 min
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 1.59 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 1.59 min




  0%|                                                 | 0/18059 [00:00<?, ?it/s]

  0%|                                      | 8/18059 [00:22<13:48:32,  2.75s/it]

  1%|▎                                      | 136/18059 [00:22<35:28,  8.42it/s]

  1%|▎                                      | 168/18059 [00:22<26:58, 11.05it/s]

  1%|▌                                      | 232/18059 [00:22<15:54, 18.68it/s]

  1%|▌                                      | 264/18059 [00:23<12:38, 23.48it/s]

  2%|▋                                      | 328/18059 [00:23<07:46, 38.02it/s]

  2%|▊                                      | 368/18059 [00:23<06:25, 45.90it/s]

  2%|▊                                      | 400/18059 [00:23<05:11, 56.61it/s]

  2%|▉                                      | 432/18059 [00:23<04:28, 65.56it/s]

  3%|▉                                      | 456/18059 [00:24<04:08, 70.75it/s]

  3%|█▏                                    | 552/18059 [00:24<02:19, 125.14it/s]

  3%|█▏       

 39%|██████████████▋                       | 6984/18059 [04:45<04:11, 43.95it/s]

 39%|██████████████▊                       | 7016/18059 [04:50<09:47, 18.80it/s]

 39%|██████████████▊                       | 7048/18059 [04:57<18:31,  9.91it/s]

 39%|██████████████▉                       | 7080/18059 [04:59<16:10, 11.31it/s]

 39%|██████████████▉                       | 7112/18059 [05:00<12:57, 14.08it/s]

 40%|███████████████                       | 7144/18059 [05:00<09:45, 18.66it/s]

 40%|███████████████                       | 7176/18059 [05:01<08:43, 20.78it/s]

 40%|███████████████▏                      | 7208/18059 [05:03<08:28, 21.34it/s]

 40%|███████████████▏                      | 7240/18059 [05:03<07:07, 25.30it/s]

 40%|███████████████▎                      | 7272/18059 [05:04<06:30, 27.64it/s]

 40%|███████████████▎                      | 7304/18059 [05:04<05:03, 35.49it/s]

 41%|███████████████▌                      | 7368/18059 [05:05<03:38, 48.88it/s]

 41%|███████████

 76%|████████████████████████████▎        | 13800/18059 [09:06<02:59, 23.71it/s]

 77%|████████████████████████████▎        | 13832/18059 [09:06<02:37, 26.81it/s]

 77%|████████████████████████████▍        | 13864/18059 [09:07<02:30, 27.91it/s]

 77%|████████████████████████████▍        | 13896/18059 [09:07<01:54, 36.32it/s]

 77%|████████████████████████████▌        | 13928/18059 [09:09<02:19, 29.56it/s]

 77%|████████████████████████████▌        | 13960/18059 [09:09<01:54, 35.67it/s]

 77%|████████████████████████████▋        | 13992/18059 [09:11<02:03, 32.80it/s]

 78%|████████████████████████████▋        | 14024/18059 [09:14<03:32, 18.98it/s]

 78%|████████████████████████████▊        | 14056/18059 [09:15<02:58, 22.41it/s]

 78%|████████████████████████████▊        | 14088/18059 [09:17<03:30, 18.89it/s]

 78%|████████████████████████████▉        | 14120/18059 [09:18<03:00, 21.77it/s]

 78%|████████████████████████████▉        | 14152/18059 [09:19<02:52, 22.65it/s]

 79%|███████████

IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 8.52 sec


...

100%|████████████████████████████████████▉| 18056/18059 [09:40<00:00, 88.44it/s]

IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 19.39 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 25.67 sec
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 33.25 sec
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 39.71 sec
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 46.00 sec
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 52.37 sec
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 59.24 sec
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 1.09 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 1.09 min




  0%|                                                 | 0/18059 [00:00<?, ?it/s]

  0%|                                      | 8/18059 [00:18<11:19:25,  2.26s/it]

  0%|                                      | 40/18059 [00:18<1:46:28,  2.82it/s]

  0%|▏                                       | 72/18059 [00:19<49:40,  6.03it/s]

  1%|▎                                      | 136/18059 [00:19<21:16, 14.04it/s]

  1%|▍                                      | 200/18059 [00:20<11:52, 25.05it/s]

  1%|▌                                      | 232/18059 [00:20<09:08, 32.50it/s]

  2%|▋                                      | 328/18059 [00:20<04:40, 63.14it/s]

  2%|▊                                      | 392/18059 [00:20<03:27, 85.34it/s]

  2%|▉                                      | 424/18059 [00:20<03:28, 84.72it/s]

  3%|█                                     | 520/18059 [00:21<02:09, 134.98it/s]

  3%|█▏                                    | 552/18059 [00:21<02:09, 135.05it/s]

  3%|█▏       

 43%|████████████████▌                     | 7848/18059 [04:13<05:38, 30.20it/s]

 44%|████████████████▌                     | 7880/18059 [04:13<04:10, 40.67it/s]

 44%|████████████████▋                     | 7912/18059 [04:13<03:08, 53.88it/s]

 44%|████████████████▋                     | 7944/18059 [04:14<03:16, 51.55it/s]

 44%|████████████████▊                     | 8008/18059 [04:14<01:59, 84.06it/s]

 45%|████████████████▌                    | 8072/18059 [04:14<01:19, 126.06it/s]

 45%|█████████████████                     | 8104/18059 [04:15<01:51, 89.45it/s]

 45%|████████████████▊                    | 8200/18059 [04:15<01:05, 150.21it/s]

 46%|████████████████▊                    | 8232/18059 [04:16<01:30, 108.90it/s]

 46%|█████████████████▍                    | 8264/18059 [04:18<03:40, 44.36it/s]

 46%|█████████████████▍                    | 8296/18059 [04:21<06:24, 25.38it/s]

 46%|█████████████████▌                    | 8328/18059 [04:31<16:33,  9.80it/s]

 46%|███████████

 82%|██████████████████████████████▎      | 14824/18059 [08:24<02:46, 19.38it/s]

 82%|██████████████████████████████▍      | 14856/18059 [08:25<02:39, 20.06it/s]

 82%|██████████████████████████████▌      | 14888/18059 [08:26<02:17, 23.13it/s]

 83%|██████████████████████████████▌      | 14920/18059 [08:26<01:44, 29.96it/s]

 83%|██████████████████████████████▋      | 14952/18059 [08:27<01:27, 35.48it/s]

 83%|██████████████████████████████▋      | 14984/18059 [08:27<01:06, 46.18it/s]

 83%|██████████████████████████████▊      | 15016/18059 [08:27<00:50, 60.63it/s]

 84%|██████████████████████████████▉      | 15080/18059 [08:28<00:38, 77.01it/s]

 84%|███████████████████████████████      | 15144/18059 [08:28<00:31, 91.16it/s]

 84%|██████████████████████████████▎     | 15208/18059 [08:29<00:28, 101.01it/s]

 84%|███████████████████████████████▏     | 15240/18059 [08:29<00:36, 76.80it/s]

 85%|███████████████████████████████▎     | 15272/18059 [08:31<00:55, 50.22it/s]

 85%|███████████

In [13]:
tuning_params = {
    "alpha":(0.8,1.0),
    "beta":(0.19,0.35)
}

In [14]:

def BO_function(alpha,beta):
    MAP = 0
    k = 5
    cutoff=10
    for i in range(len(recommenders)):
        recommenders[i].fit(alpha,beta)
        result_df, _ = evaluator_list[i].evaluateRecommender(recommenders[i])
        MAP += result_df.loc[cutoff]["MAP"]
        
    return MAP/k


In [15]:
optimizer = BayesianOptimization(f=BO_function,
                                pbounds=tuning_params,
                                verbose=5,
                                random_state=5)

optimizer.maximize(init_points=100,n_iter=100)


|   iter    |  target   |   alpha   |   beta    |
-------------------------------------------------




100%|████████████████████████████████████▉| 18056/18059 [10:16<00:00, 72.50it/s]

|  1        |  0.2913   |  0.8444   |  0.3293   |
|  2        |  0.2913   |  0.8413   |  0.337    |
|  3        |  0.291    |  0.8977   |  0.2879   |
|  4        |  0.2909   |  0.9532   |  0.2729   |
|  5        |  0.2907   |  0.8594   |  0.22     |
|  6        |  0.2911   |  0.8161   |  0.3082   |
|  7        |  0.2906   |  0.8883   |  0.2153   |
|  8        |  0.2906   |  0.976    |  0.2339   |
|  9        |  0.2906   |  0.8828   |  0.2374   |
|  10       |  0.2909   |  0.9258   |  0.2828   |
|  11       |  0.2907   |  0.92     |  0.2325   |
|  12       |  0.2906   |  0.8569   |  0.2306   |
|  13       |  0.2907   |  0.8655   |  0.2131   |
|  14       |  0.2913   |  0.8331   |  0.3442   |
|  15       |  0.2904   |  0.992    |  0.2201   |


100%|████████████████████████████████████▉| 18056/18059 [15:42<00:00, 19.16it/s]


|  16       |  0.2908   |  0.8049   |  0.2227   |
|  17       |  0.2911   |  0.94     |  0.3147   |
|  18       |  0.2912   |  0.8046   |  0.2824   |
|  19       |  0.2911   |  0.8003   |  0.2725   |
|  20       |  0.2912   |  0.928    |  0.3477   |
|  21       |  0.2912   |  0.8518   |  0.3184   |
|  22       |  0.2911   |  0.9741   |  0.3376   |
|  23       |  0.2911   |  0.8004   |  0.2651   |
|  24       |  0.2907   |  0.9963   |  0.2538   |
|  25       |  0.2909   |  0.9627   |  0.2774   |
|  26       |  0.2908   |  0.9542   |  0.2676   |
|  27       |  0.2907   |  0.8058   |  0.2038   |
|  28       |  0.2908   |  0.8223   |  0.2302   |
|  29       |  0.2909   |  0.993    |  0.2911   |
|  30       |  0.2909   |  0.9633   |  0.2806   |
|  31       |  0.2911   |  0.9271   |  0.3199   |
|  32       |  0.2911   |  0.9853   |  0.336    |
|  33       |  0.2903   |  0.965    |  0.2051   |
|  34       |  0.2904   |  0.8722   |  0.1957   |
|  35       |  0.2911   |  0.9093   |  0.3174   |


|  116      |  0.2913   |  0.8605   |  0.3274   |
|  117      |  0.2913   |  0.8067   |  0.3192   |
|  118      |  0.2912   |  0.967    |  0.3499   |
|  119      |  0.2911   |  0.8289   |  0.2886   |
|  120      |  0.2912   |  0.8      |  0.2937   |
|  121      |  0.2913   |  0.8673   |  0.3495   |
|  122      |  0.2912   |  1.0      |  0.35     |
|  123      |  0.2911   |  0.9742   |  0.3377   |
|  124      |  0.2911   |  0.9622   |  0.3321   |
|  125      |  0.2912   |  0.9534   |  0.3458   |
|  126      |  0.2911   |  0.9755   |  0.3248   |
|  127      |  0.2911   |  0.8714   |  0.295    |
|  128      |  0.2904   |  0.9189   |  0.1904   |
|  129      |  0.2911   |  0.9841   |  0.3496   |
|  130      |  0.2911   |  1.0      |  0.3367   |
|  131      |  0.2912   |  0.8352   |  0.3047   |
|  132      |  0.2909   |  0.8      |  0.2461   |
|  133      |  0.2912   |  0.8213   |  0.3497   |
|  134      |  0.2912   |  0.8129   |  0.2933   |
|  135      |  0.2913   |  0.8911   |  0.3425   |


In [17]:
import json
with open("tuning_KFOLD/RP3hSLIME_IALS/" + recommenders[0].RECOMMENDER_NAME + "_logs.json", 'w') as json_file:
    json.dump(optimizer.max, json_file)